In [1]:
import pandas as pd
from faker import Faker
import random

In [2]:
fake = Faker()

In [17]:
def generate_fake_data(num_records):
    menu_categories = {
        'Pizza': ['Margherita Pizza', 'Pepperoni Pizza', 'Vegetarian Pizza', 'Supreme Pizza', 'BBQ Chicken Pizza'],
        'Burger': ['Classic Cheeseburger', 'Mushroom Swiss Burger', 'BBQ Burger', 'Jalapeno Burger', 'Double Patty Burger'],
        'Sandwich': ['BLT Sandwich', 'Chicken Sandwich', 'Club Sandwich', 'Turkey Avocado Sandwich', 'Steak Sandwich'],
        'Pasta': ['Spaghetti Bolognese', 'Fettuccine Alfredo', 'Penne Arrabiata', 'Lasagna', 'Carbonara'],
        'Side': ['French Fries', 'Onion Rings', 'Garlic Bread', 'Coleslaw', 'Mozzarella Sticks']
        }
    dataset = []
    chosen_items = {category: [] for category in menu_categories}
    while len(dataset) < num_records:
        menu_category = random.choice(list(menu_categories.keys()))
        available_items = [item for item in menu_categories[menu_category] if item not in chosen_items[menu_category]]
        if not available_items:
            continue
        num_to_choose = min(num_records - len(dataset), len(available_items))
        chosen_items_list = random.sample(available_items, num_to_choose)
        for menu_item in chosen_items_list:
            chosen_items[menu_category].append(menu_item)
            cost = 0
            price = 0
            number_sold = random.randint(5, 60)
            if menu_category == 'Pizza':
                cost = round(random.uniform(1.50, 3.00), 2)
                price = random.randint(8, 14)
            elif menu_category == 'Burger':
                cost = round(random.uniform(1.50, 2.50), 2)
                price = random.randint(7, 11)
            elif menu_category == 'Sandwich':
                cost = round(random.uniform(2, 4), 2)
                price = random.randint(5, 8)
            elif menu_category == 'Pasta':
                cost = round(random.uniform(2, 5), 2)
                price = random.randint(9, 15)
            elif menu_category == 'Side':
                cost = round(random.uniform(0.50, 2.00), 2)
                price = random.randint(3, 5)
            
            contribution_margin = price - cost
            
            record = {
                'Menu Item': menu_item,
                'Menu Category': menu_category,
                'Cost': cost,
                'Price': price,
                'Number Sold': number_sold,
                'Contribution Margin': contribution_margin
            }
            dataset.append(record)
            
    return dataset

In [18]:
# Generate 20 fake records
fake_data = generate_fake_data(25)

In [19]:
# Create a Pandas DataFrame from the fake data
df = pd.DataFrame(fake_data)

In [20]:
# Print the DataFrame
df.head()

,Menu Item,Menu Category,Cost,Price,Number Sold,Contribution Margin
0,Steak Sandwich,Sandwich,3.31,7,57,3.69
1,Chicken Sandwich,Sandwich,3.30,5,58,1.70
2,BLT Sandwich,Sandwich,3.80,7,17,3.20
3,Turkey Avocado Sandwich,Sandwich,3.50,7,21,3.50
4,Club Sandwich,Sandwich,3.59,8,23,4.41


In [21]:
# Write to an excel 
df.to_excel('./data/menu_data.xlsx', index=False)

# Visualization 1

In [16]:
import plotly.express as px

# Define thresholds for profitability and popularity
profitability_threshold = df['contribution_margin'].mean()
popularity_threshold = df['number_sold'].mean()

# Assign labels based on profitability and popularity categories
df['label'] = ''
df.loc[(df['contribution_margin'] > profitability_threshold) & (df['number_sold'] > popularity_threshold), 'label'] = 'Star'
df.loc[(df['contribution_margin'] > profitability_threshold) & (df['number_sold'] <= popularity_threshold), 'label'] = 'Puzzle'
df.loc[(df['contribution_margin'] <= profitability_threshold) & (df['number_sold'] > popularity_threshold), 'label'] = 'Plow-Horse'
df.loc[(df['contribution_margin'] <= profitability_threshold) & (df['number_sold'] <= popularity_threshold), 'label'] = 'Dog'

# Create a scatter plot using Plotly Express
fig = px.scatter(df, x='contribution_margin', y='number_sold', color='label', text='menu_item', size_max=10)

# Customize the plot
fig.update_traces(textfont=dict(color='black', size=10), hovertemplate='<b>%{text}</b>')
fig.update_layout(title='Profitability vs. Popularity',
                  xaxis_title='Profitability',
                  yaxis_title='Popularity',
                  width=800,  # Adjust the width
                  height=500,  # Adjust the height
                  template='simple_white',  # Set the template to 'simple_white'
                  margin=dict(t=100, b=50, l=50, r=50))  # Add margins to the plot

# Set legend properties
fig.update_layout(
    legend=dict(
        title=None,
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5,
        itemclick=False
    )
)

# Adjust label positions to be below the data points
fig.update_traces(textposition='bottom center')

# Show the plot
fig.show()


# Visualization 2

In [17]:
import plotly.express as px

# Define thresholds for profitability and popularity
profitability_threshold = df['contribution_margin'].mean()
popularity_threshold = df['number_sold'].mean()

# Assign labels based on profitability and popularity categories
df['label'] = ''
df.loc[(df['contribution_margin'] > profitability_threshold) & (df['number_sold'] > popularity_threshold), 'label'] = 'star'
df.loc[(df['contribution_margin'] > profitability_threshold) & (df['number_sold'] <= popularity_threshold), 'label'] = 'puzzle'
df.loc[(df['contribution_margin'] <= profitability_threshold) & (df['number_sold'] > popularity_threshold), 'label'] = 'plow-horse'
df.loc[(df['contribution_margin'] <= profitability_threshold) & (df['number_sold'] <= popularity_threshold), 'label'] = 'dog'

# Abbreviate data point text labels
df['abbreviated_menu_item'] = df['menu_item'].str[:5] + '...'

# Create a scatter plot using Plotly Express
fig = px.scatter(df, x='contribution_margin', y='number_sold', color='label', text='abbreviated_menu_item', size_max=10)

# Customize the plot appearance
fig.update_traces(
    textfont=dict(color='black', size=12),  # Increase the font size
    hovertemplate='<b>%{text}</b>',
    textposition='bottom center'  # Position the text labels at the bottom center
)
fig.update_layout(
    title='Profitability vs. Popularity',
    xaxis_title='Profitability',
    yaxis_title='Popularity',
    width=800,
    height=500,
    template='plotly_white',  # Use a simple white template
    font_family='Arial',
    font_color='black',
    legend=dict(
        orientation='h',
        yanchor='top',
        y=1.1,  # Adjust the position
        xanchor='center',
        x=0.5,
        itemclick=False,
        title=None  # Remove the legend title
    ),
    margin=dict(t=80, b=50, l=50, r=50),
    xaxis=dict(
        zeroline=True,  # Show the x-axis line
        zerolinecolor='gray',  # Set the color of the x-axis line
        zerolinewidth=1,  # Set the width of the x-axis line
        showgrid=False  # Remove the x-axis gridlines
    ),
    yaxis=dict(
        zeroline=True,  # Show the y-axis line
        zerolinecolor='gray',  # Set the color of the y-axis line
        zerolinewidth=1,  # Set the width of the y-axis line
        showgrid=False  # Remove the y-axis gridlines
    ),
    shapes=[
        dict(
            type="line",
            x0=min(df['contribution_margin']),
            y0=df['number_sold'].mean(),
            x1=max(df['contribution_margin']),
            y1=df['number_sold'].mean(),
            line=dict(color='gray', width=1),
            opacity=0.3,
            layer='below'
        ),
        dict(
            type="line",
            x0=df['contribution_margin'].mean(),
            y0=min(df['number_sold']),
            x1=df['contribution_margin'].mean(),
            y1=max(df['number_sold']),
            line=dict(color='gray', width=1),
            opacity=0.3,
            layer='below'
        )
    ]
)

# Show the plot
fig.show()

# Visualization 3 

In [18]:
import pandas as pd
import plotly.graph_objects as go

# Group menu items by category and label
grouped_data = df.groupby(['menu_category', 'label'])['menu_item'].apply(lambda x: ', '.join(x)).unstack()

# Reindex to include all categories
categories = df['menu_category'].unique()
grouped_data = grouped_data.reindex(categories, fill_value='')

# Create the table
table = go.Table(
    header=dict(values=['Menu Category', 'Star', 'Plow-Horse', 'Puzzle', 'Dog']),
    cells=dict(values=[grouped_data.index, grouped_data['star'], grouped_data['plow-horse'], grouped_data['puzzle'], grouped_data['dog']])
)

# Define the layout
layout = go.Layout(
    title='Menu Item Categorization',
    width=800,
    height=500
)

# Create the figure and add the table
fig = go.Figure(data=[table], layout=layout)

# Show the table
fig.show()
